# BIWENGER ANALYSIS

This script obtains the data from Biwenger and analyzes different features that could be useful to become better at this fantasy game. For example, the posibilities of one player to be on the starting eleven, the chances of getting certain players on a pack...

## Data Extraction

In [ ]:
import requests
import csv
import pandas as pd

#Obataining the Biwenger json

r = requests.get(
    "https://cf.biwenger.com/api/v2/competitions/la-liga/data?lang=en&score=1").json()

#Extracting the players data and storing it as csv

data = []
for k, v in r['data']['players'].items():
    data.append(v)

with open('output.csv', 'w', newline="", encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(v.keys())
    writer.writerows(data)
    
df = pd.DataFrame(data)

In [ ]:
#Show the dataset
df

## Number of Players in a Price Range

In [ ]:
n_players = df['price'].count()
precios = list(df['price'])

In [ ]:
def rango_precios(a,b,precios):
    count = 0
    for i in range (len(precios)):
        if precios[i]>=a and precios[i]<=b:
            count = count + 1
    porcentaje = (count/len(precios))*100
    return (count, porcentaje)
    

In [ ]:
porcentaje = rango_precios(150000,500000,precios)[1]
print('The percentage of players on this price interval is :', porcentaje, '%')

## Histogram of Players in a Price Range

In [ ]:
import matplotlib.pyplot as mp
intervalos = range(min(precios),max(precios),500000)
mp.hist(x=precios, bins=intervalos, color='#F2AB6D', rwidth=0.85)
mp.title('Prices Histogram')
mp.xlabel('Price')
mp.ylabel('Players')

## TOP 10 Price Increase of Players

In [ ]:
delta_max=[]
player_delta=[]
for i in range (len(df['priceIncrement'])):
    if df['priceIncrement'][i]>=80000:
        delta_max.append(df['priceIncrement'][i])
        player_delta.append(df['name'][i])
        
price_var = dict(zip(player_delta,delta_max))

In [ ]:
import operator
price_var = sorted(price_var.items(), key=operator.itemgetter(1))
price_var

## PACK PROBABILITIES

This script calculate the probability of unpacking a player on a scpecific range of prices when buying one of the Biwenger packs.

## Bronze Pack

In [ ]:
probability = []
rango = []
n=0
p=0
bronze_players = rango_precios(0,5000000,precios)[0]
for i in range (1000000,5000001,1000000):
    z = str(i-1e6) + ' - ' + str(i)
    rango.append(z)
    n = rango_precios(i-1000000,i,precios)[0]
    p = (n/bronze_players)*100
    probability.append(p)

probabilidad = dict(zip(rango,probability))

keys = []
values = []
for key,value in probabilidad.items():
    keys.append(str(key))
    values.append(str(value))

s= dict(zip(keys,values))
d = pd.DataFrame(s, index = ['Probabilidad (%)'])
d

## Silver Pack

In [ ]:
probability = []
rango = []
n=0
p=0
silver_players = rango_precios(5000000,10000000,precios)[0]
for i in range (6000000,10000001,1000000):
    z = str(i-1e6) + ' - ' + str(i)
    rango.append(z)
    n = rango_precios(i-1000000,i,precios)[0]
    p = (n/silver_players)*100
    probability.append(p)
    
invest_recover = probability[3]+probability[4]
print ('Las posibilidades de ganar dinero con el sobre son de: ',invest_recover)
probabilidad = dict(zip(rango,probability))

keys = []
values = []
for key,value in probabilidad.items():
    keys.append(str(key))
    values.append(str(value))

s= dict(zip(keys,values))
d = pd.DataFrame(s, index = ['Probabilidad (%)'])
d

In [ ]:
probability

## Gold Pack

In [ ]:
probability = []
n=0
p=0
gold_players = rango_precios(10000000,30000000,precios)[0]
for i in range (10000000,30000001,1000000):
    z = str(i-1e6) + ' - ' + str(i)
    rango.append(z)
    n = rango_precios(i-1000000,i,precios)[0]
    p = (n/gold_players)*100
    probability.append(p)

probabilidad = dict(zip(rango,probability))

keys = []
values = []
for key,value in probabilidad.items():
    keys.append(str(key))
    values.append(str(value))

s= dict(zip(keys,values))
d = pd.DataFrame(s, index = ['Probabilidad (%)'])
d

## STARTING ELEVEN CHANCES

In [ ]:
from bs4 import BeautifulSoup
import urllib
import re

url = 'https://www.futbolfantasy.com/'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html)
teams = soup('a',class_ ='team')
Equipos = []
for team in teams:
    alt = team.get('alt')
    if alt not in Equipos:
        Equipos.append(alt)

### START THE CRAWLER ###

links=[]
for team in teams:
    link = team.get('href')
    if link not in links:
        links.append(link)


titulares_equipos=[]
for url2 in links:
    html2 = urllib.request.urlopen(url2)
    soup2 = BeautifulSoup(html2)
    equipo = soup2('span', class_='nombre')
    equipo = re.findall('(?<=>).*(?=<)',str(equipo))
    jug = soup2('a', class_='juggador')
    datos_jug=[]
    for i in jug:
        datos_jug.append(i.contents)
        p = re.findall('\d\d%',str(i.contents))
        n = re.findall('(?<=>).*(?=<)',str(i.contents))
        nombres_titulares = []
        for nombres in titulares_equipos:
            nombres_titulares.append(nombres[0])
        if len(n)>0:
            if n[0] not in nombres_titulares:
                if p != []:
                    z = [n[0],p[0],equipo[0]]
                    titulares_equipos.append(z)

In [ ]:
df_ff = pd.DataFrame(titulares_equipos,columns=['name','Titularidad','Equipo'])
df_ff

In [ ]:
#Get the starting eleven chances:
def titularidad_equipo(equipo):
    print(df_ff.get(df_ff['Equipo']==equipo))

In [ ]:
titularidad_equipo('Real Madrid')

## API - SOFASCORE

### Sofascore Average Score

In [ ]:
import requests
import json
ids=[]
equipo_ids=dict()
url = "https://sofascore.p.rapidapi.com/teams/search"
Sofascore=pd.DataFrame(columns=['Jugador','Media','Equipo','ID'])
for equipo in Equipos:
    try:
        url = "https://sofascore.p.rapidapi.com/teams/search"
        querystring = {"name":equipo}
        headers = {
            'x-rapidapi-host': "sofascore.p.rapidapi.com",
            'x-rapidapi-key': "##########################################"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        r=json.loads(response.text)

        ids.append(r['teams'][0]['id'])
        equipo_ids[equipo]=r['teams'][0]['id']

        url = "https://sofascore.p.rapidapi.com/teams/get-squad"
        for id_ in ids:

            querystring = {"teamId":id_}

            headers = {
                'x-rapidapi-host': "sofascore.p.rapidapi.com",
                'x-rapidapi-key': "##########################################"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)
            r=json.loads(response.text)

            p=r['players']
            for i in range(len(p)):
                idd=p[i]['player']['id']
                player=p[i]['player']['name']
                url = "https://sofascore.p.rapidapi.com/players/get-last-ratings"

                querystring = {"playerId":idd,"tournamentId":"8","seasonId":"37223"}

                headers = {
                    'x-rapidapi-host': "sofascore.p.rapidapi.com",
                    'x-rapidapi-key': "##########################################"
                    }

                response = requests.request("GET", url, headers=headers, params=querystring)
                r=json.loads(response.text)
                ratings = r['lastRatings']
                j=0
                rates=0
                try:
                    for i in range(len(ratings)):
                        rate=ratings[i]['rating']
                        j=j+1
                        rates=rates+rate
                    media=rates/j
                    Sofascore=Sofascore.append({'Jugador':player,'Media':media,'Equipo':equipo,'ID':idd},ignore_index=True)
                except:
                    continue
    except:
        print('Error en la extracción de datos del equipo:',equipo)

In [ ]:
## Show the Sofascore Dataset
Sofascore